# Import Dependencies

In [4]:
import openai

# What is OpenAI API?

The OpenAI API refers to the application programming interface provided by OpenAI, a leading artificial intelligence research lab. The OpenAI API allows developers to access and integrate OpenAI's powerful language models into their own applications, products, or services.

One notable model offered through the OpenAI API is GPT (Generative Pre-trained Transformer). GPT models are designed for natural language processing tasks, such as text generation, completion, translation, summarization, and more. The API provides a way for developers to interact with these models and leverage their capabilities without having to train the models from scratch.

Keep in mind that the specifics of the OpenAI API, including its features and offerings, may have evolved since my last update in January 2022. Therefore, it's a good idea to check the official OpenAI website or other reliable sources for the most up-to-date information on the OpenAI API.


# Generate OPEN API Key

In [5]:
Myapi = ""

In [6]:
openai.api_key = Myapi

In [9]:
All_Models = openai.models.list()

In [10]:
list(All_Models)

[Model(id='gpt-3.5-turbo-0613', created=1686587434, object='model', owned_by='openai'),
 Model(id='curie-search-query', created=1651172509, object='model', owned_by='openai-dev'),
 Model(id='babbage-search-query', created=1651172509, object='model', owned_by='openai-dev'),
 Model(id='dall-e-3', created=1698785189, object='model', owned_by='system'),
 Model(id='babbage-search-document', created=1651172510, object='model', owned_by='openai-dev'),
 Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-0301', created=1677649963, object='model', owned_by='openai'),
 Model(id='gpt-3.5-turbo-16k-0613', created=1685474247, object='model', owned_by='openai'),
 Model(id='text-embedding-ada-002', created=1671217299, object='model', owned_by='openai-internal'),
 Model(id='davinci-search-query', created=1651172505, object='model', owned_by='openai-dev'),
 Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='system'),
 Model(id='gpt-

In [12]:
import pandas as pd

In [14]:
pd.DataFrame(list(All_Models), columns = ["ID","Created", "Object","Owned_By"])

,ID,Created,Object,Owned_By
0,"(id, gpt-3.5-turbo-0613)","(created, 1686587434)","(object, model)","(owned_by, openai)"
1,"(id, curie-search-query)","(created, 1651172509)","(object, model)","(owned_by, openai-dev)"
2,"(id, babbage-search-query)","(created, 1651172509)","(object, model)","(owned_by, openai-dev)"
3,"(id, dall-e-3)","(created, 1698785189)","(object, model)","(owned_by, system)"
4,"(id, babbage-search-document)","(created, 1651172510)","(object, model)","(owned_by, openai-dev)"
5,"(id, dall-e-2)","(created, 1698798177)","(object, model)","(owned_by, system)"
6,"(id, gpt-3.5-turbo-0301)","(created, 1677649963)","(object, model)","(owned_by, openai)"
7,"(id, gpt-3.5-turbo-16k-0613)","(created, 1685474247)","(object, model)","(owned_by, openai)"
8,"(id, text-embedding-ada-002)","(created, 1671217299)","(object, model)","(owned_by, openai-internal)"
9,"(id, davinci-search-query)","(created, 1651172505)","(object, model)","(owned_by, openai-dev)"


## Code of OpenAI Chat

In [16]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "user",
      "content": "How I can Make Money?"
    }
  ],
  temperature=1,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

## This is where you make a request to the OpenAI API for generating a chat completion. Let's break down the parameters:

    model="gpt-3.5-turbo": Specifies the language model to be used, in this case, GPT-3.5-turbo.

    messages: A list of message objects representing the conversation. In this example, there is only one message from the user: "How I can Make Money?".

    temperature=1: A parameter controlling the randomness of the model's output. Higher values like 1 make the output more random, while lower values like 0.2 make it more focused and deterministic.

    max_tokens=256: Specifies the maximum number of tokens (words or characters) in the generated response.

    top_p=1: Top-p (nucleus) sampling probability. Setting it to 1 means the model considers the entire probability distribution for sampling.

    frequency_penalty=0: Controls the penalty for using frequent tokens. A higher value will make the model more creative, while a lower value will make it more conservative.

    presence_penalty=0: Controls the penalty for using new tokens. Similar to frequency_penalty, but for unseen tokens.